<a href="https://colab.research.google.com/github/rzl-ds/gu511_hw/blob/master/hw04.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg"/>
</a>

# Exercises due by EOD 2020.09.24

## goal

in this homework assignment we will continue exploring `ssh` and `linux` skills, and start to `git` more familiar with `git`

## method of delivery

*as mentioned in our first lecture, the method of delivery may change from assignment to assignment. we will include this section in every assignment to provide an overview of how we expect homework results to be submitted, and to provide background notes or explanations for "new" delivery concepts or methods.*

this week you will be submitting the results of your homework via **EMAIL** and **GITHUB**

+ subject: "hw05 answers"
+ to: rzl5@georgetown.edu, ip221@georgetown.edu

summary:

| exercise | deliverable | method of delivery | points |
|----------|-------------|--------------------|--------|
| 1 | public ip address of `ec2` instance we can `ssh` into | include in body of submission email | 10 |
| 2 | none | none | 5 |
| 3 | a `push` of a first commit of a local repo to a shared `github` repo | none (we will get an email) | 5 |
| 4 | none | none | 3 |
| 5 | a commit containing `helloworld.py` | `push` to your `github` repo | 5 |
| 6 | a commit containing changes to `README.md` | `push` to your `github` repo | 5 |
| 7 | none | none | 2 |


total points: 35

<div style="border: 1px solid lightgrey;">

# exercise 1: set up *MY* login to *your* `ec2` instance

we now have three computers in place -- your laptop, your `ec2` instance, and my `ec2` instance. we've set up connections

+ your laptop `-->` your `ec2`
+ your laptop `-->` my `ec2`

I gave you the ability to log in to my `aws` instance, and now it's time to repay the favor. we will set up one final connection

+ my `ec2` `-->` your `ec2`

being knowledgable in the ways of `ssh`, you know the information you require, so you ask me:

1. what is the IP address from which you will be connecting?
1. what is your private key?

and I respond:

1. `3.223.216.195`
1. [pretty public all right](https://gist.githubusercontent.com/RZachLamberty/2c29bb3d6ce77862d113e967b43fe2d7/raw/bed41d1efed6c6fe8feb4be570054e3ac56b360a/zachs_ec2.gu511_ssh_hw.2020.pub)

so what now?

## 1.1: configure the `aws` security group to allow traffic from my ip address

first, you have to make sure I am *allowed* to send `ssh` messages. you do that in the `aws` `ec2` console (remember that?!).

[log in to `aws`](https://console.aws.amazon.com/) and select the `ec2` service and then the "instances" sub-menu. select your `ec2` instance so that the description panel fills in on the bottom. then, click the link to visit your `ec2` instance's security group

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1jNTmBuwqRNaVNBQQYaotT-0b90GQSZS5"></div>

in the modern view, this will look like

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1EiAyay9ou9sl2lMdg1wTcENyPIuThRG4"></div>

once there, click on the "inbound" tab in the lower panel, and verify that one of the two following statements is true:

1. you have a rule allowing `tcp` traffic on port 22 (e.g. `ssh`) from *all* ip addresses (`cidr = 0.0.0.0/0`), or
1. you have a rule allowing `tcp` traffic on port 22 (e.g. `ssh`) from *my* ip address (`cidr = 3.223.216.195/32`)

## 1.2: set up my user on your `ec2` instance

the next step is for you to create a new user on your `ec2` instance. this is an administrator level task, so we must become a super user -- using `sudo`.

the command we will execute is the `useradd` command which -- surprise, surprise -- adds users.

the `-m` flag creates a home directory for me.

run the following command to create a new user named `rzl5` on your machine:

```sh
sudo useradd  -s /bin/bash -m rzl5
```

## 1.3: add the public key for my user

for me to log in as new user `rzl5`, that user must have my public key in a `.ssh` folder in that user's home directory. you can make that happen!

to start, give my user a `/home/rzl5/.ssh` folder:

```sh
# create a .ssh folder for my user, and change the owner
sudo mkdir -p /home/rzl5/.ssh
sudo chown rzl5:rzl5 /home/rzl5/.ssh
```

now, we need to add the public key to a file called `authorized_keys` in that `~/.ssh` directory execute the commands below, but replace the `REPLACE_THIS_PUB_KEY_STRING` with the value on the [`github` gist](https://gist.githubusercontent.com/RZachLamberty/2c29bb3d6ce77862d113e967b43fe2d7/raw/bed41d1efed6c6fe8feb4be570054e3ac56b360a/zachs_ec2.gu511_ssh_hw.2020.pub). **leave the quotations around the very long public key!":

```sh
# add public key to authorized_keys and then fix permissions
sudo sh -c 'echo "REPLACE_THIS_PUB_KEY_STRING" >> /home/rzl5/.ssh/authorized_keys'

# change the owner of my .ssh directory to be me,
# then set the permission MODe to be restricted to
# me alone
sudo chown -R rzl5:rzl5 /home/rzl5/.ssh/
sudo chmod 700 /home/rzl5/.ssh
sudo chmod 600 /home/rzl5/.ssh/authorized_keys
```

you can verify you've done this all correctly if you run the following command and see a similar output (not necessarily the file size, but definitely the permission mode strings):

```sh
ubuntu@ip-XXX-XXX-XXX-XXX:~$ sudo ls -alh /home/rzl5/.ssh
```

```
total 12K
drwx------ 2 rzl5 rzl5 4.0K Sep 16 18:46 .
drwxr-xr-x 4 rzl5 rzl5 4.0K Sep 21 00:05 ..
-rw------- 1 rzl5 rzl5  740 Sep 16 18:46 authorized_keys
```

## 1.4: send me your `ec2` instance's public ip address or DNS

finally, send me the *public ip address* or *public DNS* of your instance, and make sure it stays on long enough for me to make an ssh connection.


##### email us your public ip address

<div style="border: 1px solid lightgrey;">

# exercise 2: create a brand new *local* repository

we are going to create a directory with a single file (a `README.md` file) and we will start tracking it with version control.

1. choose to use your home laptop or `ec2` instance. you can use either, but you will have to edit files and use `git` in that place you choose for the rest of the class
    1. if you're not sure, I recommend you use your laptop
1. in some place you will not forget, create a directory named `gu511_git_hw`
1. move into that directory (`cd`)
1. run the following command: `echo "# 511 github repo" > README.md`

now the goal is to create a local `git` repository that can track changes in this new `gu511_git_hw` directory. do the following:

1. `init`ialize a `git` repository in this directory
1. `add` the `README.md` to the `git` repository for staging
1. `commit` this addition: run `git commit -m 'gu511 git hw: initial commit'`

##### nothing to submit here; we will see it when you `push` to `github` in a later exercise

<div style="border: 1px solid lightgrey;">

# exercise 3: connect your *local* and *remote* `gu511_git_hw` repos

in exercise 1 we created a `local` repository called `gu511_git_hw`, and on a previous homework assignment we created an empty repository on `github` with that same name `gu511_git_hw` -- what a coincidence!

right now, on your `github` repository home page there is a set of 3 commands under the title **"...or push an existing repository from the command line"**.

select the `ssh` option at the top of that page and copy those commands; they should be something like

```sh
git remote add origin git@github.com:YOUR_GITHUB_USER_NAME>/gu511_git_hw.git
git branch -M master
git push -u origin master
```

run those commands to add your `github` repo as the *remote* for your *local* repo, and then `push` your single `commit` message to `github`

*note: if you get an authentication error message when you try to run the `git push -u origin mater` command, you have not yet set up `ssh` authentication with `github`. go read that section in the `git` lecture and set up `ssh` authentication and try again

##### we will get an email that this has been done -- we are collaborators, after all

<div style="border: 1px solid lightgrey;">

# exercise 4: add several `git` aliases

like many `linux` tools, `git` allows you to configure *aliases* (shortcut phrases for invoking long strings of commands). there are a number of these aliases that I find routinely useful, and we are going to add them to your `git` config files.

I'd recommend doing the following on all the computers on which you intend to use `git` (`ec2`, personal laptop, etc). for now, do it at least on the computer where you created your `gu511_git_hw` repository.

## 4.1: locating your `git` config file

remember our frequent mantra about `linux`-world command configuration: you can configure a command

1. *one-off*: on the command line, when you invoke that command
1. *local*: with files saved in the working directoy in which you invoke that command
1. *global*: with files save in your home directory
1. *system*: with files saved in `/etc`

we are going to update that 3rd item -- the global `git` config, a file that is saved in your home directory (it is "global" in that you can use it anywhere you are, it applies globally for you)

in `mac`, `linux`, and `windows` with `git bash`, `git` will look for a file at the path

```sh
~/.gitconfig
```

for `mac` and `linux`, this is all you need to know. for `windows`, `~` is *usually* `C:\Users\YOUR_USER_NAME`. that being said, you should verfiy by running (in `git bash`, not in `cmd` or `ps`)

```sh
cd ~ && pwd
```

to see what `git bash` sees as your `~` directory on your windows machine.

## 4.2: adding a few aliases from the command line

the `git config` command allows you to set configuration values via the command line -- see [the documentation here](https://git-scm.com/book/en/v2/Git-Basics-Git-Aliases). let's set the first few aliases -- two-letter shorthands for the commands we learned yesterday -- using this command. run the following from your terminal:

```sh
git config --global alias.co checkout
git config --global alias.br branch
git config --global alias.ci commit
```

## 4.3: editing that file

open `~/.gitconfig` in the text editor of your preference and verify that those aliases were added somewhere within it:

```config
[alias]
    ci = commit
    br = branch
    co = checkout
```

the format of this file is a common format (often called [`INI` formatting](https://en.wikipedia.org/wiki/INI_file)) in which configuration variables are grouped by concept (e.g. `alias` variables) into *sections*. the name of the section is written inside braces (`[section name]`). the configuration variables are set via `key = value` statements, often indented.

update the block under the `[alias]` heading to look like:

```config
[alias]
    ci = commit
    br = branch
    co = checkout
    st = status -sb
    gr = log --graph --abbrev-commit --decorate --date=relative --format=format:'%C(bold blue)%h%C(reset) - %C(bold green)(%ar)%C(reset) %C(bold white)%s%C(reset) - %C(red)%an%C(reset)%C(bold yellow)%d%C(reset)' --all
    unstage = reset HEAD --
```

try `git st` and `git gr` to see what they do!

a lot of folks share their `.gitconfig` files online so look around for configurations you like. [here's mine](https://github.com/RZachLamberty/linux_configs/blob/master/files/.gitconfig)

<div style="border: 1px solid lightgrey;">

# exercise 5: add a new `python` file to your local `gu511_git_hw` repo and `push` to your remote

<span style="color:red;font-weight:bold">make sure you have finished exercise 3 before starting this one. to verify that you have, run the following commands and look for the corresponding outputs</span>

| command to run | expected output |
|-|-|
| `git log --oneline` | `dcc3742 (HEAD -> master) gu511 git hw: initial commit` (your sha will differ) |
| `git remote` | `origin` |


write the following code to a file named `helloworld.py` in your local `gu511_git_hw` directory (that is, save it next to `README.md`):

```py
import datetime

def hello_world():
    print('hello world!')
    now = datetime.datetime.now()
    print(f'it is {now}')

if __name__ == "__main__":
    hello_world()
```

after editing,

1. `add` this new file to your repository
1. `commit` this new file with a message `helloworld: initial commit`
1. `push` your chagnes to `github`

<div style="border: 1px solid lightgrey;">

# exercise 6: edit the `README.md` and `push`

<span style="color:red;font-weight:bold">make sure you have finished exercise 5 before starting this one</span>

edit the existing `README.md` to read:

```
# 511 github repo

the primary function of this repo is to develop `git` skills over the course of the year.

## repository contents

+ `helloworld.py`
    + run with `python helloworld.py`
    + this will greet you and then tell you the current time
```

when you commit, use the message `README: adding a contents section`. then `push` these results to `github`

<div style="border: 1px solid lightgrey;">

# exercise 7: consider joining a local data science meetup

there are a lot of good DC-area meetup options for people interested in data science, and I strongly recommend you look into the different groups in the area to identify topics you are interested in. in addition to being a long-term way of keeping up to date on what other folks in the community are doing, it's an open community for you to find smart people who are doing the sorts of things you are or would like to do in the future.

one silver lining to the pandemic cloud: remote sessions of meetups have made it much easier to attend virtually.

in particular, I recommend the following:

+ [Data Community DC](http://www.datacommunitydc.org/meetups-overview/). this is an umbrella organization covering a ton of the local data science organization and meetup scene. there are currently 10 sub-meetup groups focused on different topics, and I've found that the quality of the individual presentations varies but is generally more worth attending than not
+ [DataKind DC](http://www.meetup.com/DataKind-DC/): (**note: this group has paused their regular meetups, but still does some projects at this time**) this is a really interesting group which works with nonprofits in the area to identify data science problems that could be of use to those organizations. There is a team of professional volunteers who work with the nonprofits before it ever gets to you, the data scientisit volunteers. this means that the projects that you are walking into are usually *at least* scoped and vetted, leaving you with something tangible to look into. you and a dozen or so other data scientists will then help that nonprofit put together a data science solution a real problem they have.
    + this is a *great* way to get experience working on a team on a scoped and defined data science problem. in my experience, whether you see success, failure, or something inbetween, this is **invaluable experience for future interviews** for real data science positions
    + there are multiple projects to pick from each meeting so you can see if any of the projects speak to you technically or personally.
    + an example project: they worked with the american red cross to develop a model to predict risk due to lack of smoke alarms in homes
+ [Code for DC](https://codefordc.org/): this is much like DataKind above, but less specifically focused on data science and more focused on projects affecing DC. if you are interested in coding generally, or seeing if you can find a good applciation of your data science skills in these settings, I highly recommend this friendly group. they will take on folks of *all* experience levels, especially including `none`. they're happy to teach, in my experience.


##### there is nothing to deliver here

<div style="border: 1px solid lightgrey;">